# TensorFlow Lite model for deployment

The TFLite model can be deployed on cloud platforms as a web service via REST API, allowing for larger inputs and more complex models to be processed, it also enables scaling.

## Convert Keras model to TFLite

We have created a model using Keras and now we are going to convert it into TensorFlow Lite (TFLite) format to use it in a web application. This conversion process involves several steps to adapt the model to this specific format.

In [26]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('../models/xception_v5_01_0.985.h5')

# Initialize tf-lite converter for keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Make model conversion
tflite_model = converter.convert()

# Save the model in tflite format
with open('../models/wildfire-model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\awon\AppData\Local\Temp\tmpgdvfccd3\assets


INFO:tensorflow:Assets written to: C:\Users\awon\AppData\Local\Temp\tmpgdvfccd3\assets


## Removing TensorFlow dependencies

We want to reduce the size of our TensorFlow model for deployment by removing unnecessary TensorFlow dependencies using TensorFlow Model Optimization Toolkit or TensorFlow Lite's tflite_convert tool, which can prune, quantize and compress the model, resulting in a smaller and more efficient model.

A simpler way of loading and preprocessing image is using [keras-image-helper](https://pypi.org/project/keras-image-helper/), also, our model needs tf-lite from tensorflow package, however, we can use tf-lite without depending on on tensorflow and this is what we are looking for. For this, we need to install `tflite-runtime` from GitHub Coral Python page.

A more straightforward method for loading and preparing images is by utilizing the [keras-image-helper](https://pypi.org/project/keras-image-helper/) package. Additionally, our model requires the `tf-lite` module from the Tensorflow library. However, it is possible to use `tf-lite` without depending on the Tensorflow package, which is what we are aiming for. To achieve this, we must install the tflite-runtime from the [GitHub Coral Python](https://github.com/google-coral/py-repo) page. `tflite-runtime` can be installed using `pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite-runtime`

In [1]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

## Make predictions with tflite model

To make predictions using TensorFlow Lite (TFLite), you must first load a pre-trained TFLite model using the TFLite Interpreter API. Then, you can use the interpreter to run inference on your input data, which should match the shape and type expected by the model. The interpreter will return the output in the form of a multi-dimensional array, containing the model's predictions for the input data.

In [2]:
# Instantiate model using Interpreter class
interpreter = tflite.Interpreter(model_path='../models/wildfire-model.tflite')
# Load the weights from the model to memory
interpreter.allocate_tensors()

# Get the input index from interpreter
input_index = interpreter.get_input_details()[0]['index']
# Get the output index from interpreter
output_index = interpreter.get_output_details()[0]['index']

In [3]:
# Create preprocessor for the model we used for training (Xception)
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [4]:
# Preprocess the image (from path)
X_path = preprocessor.from_path('../assets/land_img.jpg')

# Preprocess the image (from url)
X_url = preprocessor.from_url('https://ichef.bbci.co.uk/news/976/cpsprodpb/32A5/production/_123456921_australianbushfire_gettyimages-1198540877.jpg')

In [5]:
# Make prediction on image from path
interpreter.set_tensor(input_index, X_path)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)[0]

# List of classes the model was trained to predict
classes = ['fire', 'nofire']

# Check the prediction scores
dict(zip(classes, preds))

{'fire': 0.0, 'nofire': 1.0}

In [6]:
# Make prediction on image from url ()
interpreter.set_tensor(input_index, X_url)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)[0]

# Check the prediction scores
dict(zip(classes, preds))

{'fire': 1.0, 'nofire': 0.0}